# AN2DL - Second Challenge

Lorenzo Bardelli, Lorenzo Moretti, Luca Zani

⁠`thegradientdescenders`

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

# Set seed for reproducibility
SEED = 123

# Set debug for showing extra outputs
DEBUG = False

# Import necessary libraries
import os
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
from torch.nn import functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision.transforms import v2 as transforms
from torch.utils.data import TensorDataset, DataLoader


if IN_COLAB:
    !pip install transformers
from transformers import AutoModel

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.manual_seed(SEED)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

import cv2
import matplotlib.pyplot as plt

# Configure plot display settings
plt.rc('font', size=14)
%matplotlib inline

In [ ]:
# Define augmentation and normalization for datasets

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

class AugmentDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, augmentation=None, normalize: str = None):
        # Convert to tensor and permute to (N, C, H, W)
        self.data = torch.from_numpy(data).permute(0, 3, 1, 2)
        self.labels = torch.from_numpy(labels).long()
        self.augmentation = augmentation
        self.normalize = normalize

        # ImageNet normalization transform
        if normalize == "imagenet":
            self.normalize = transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        else:
            self.normalize = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx].clone()  # Clone to avoid modifying original
        label = self.labels[idx]

        if self.augmentation:
            image = self.augmentation(image)
        
        if self.normalize:
            image = self.normalize(image)  # Normalize only RGB channels

        return image, label

In [ ]:
class PhikonTransferLearning(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.3, freeze_backbone=True, unfreeze_last_n_layers=0):
        super().__init__()
        self.backbone = AutoModel.from_pretrained("owkin/phikon")
        self.hidden_size = self.backbone.config.hidden_size

        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
            # Unfreeze LayerNorm for domain adaptation
            for module in self.backbone.modules():
                if isinstance(module, nn.LayerNorm):
                    for param in module.parameters():
                        param.requires_grad = True

            # Unfreeze last N transformer layers for fine-tuning 
            if unfreeze_last_n_layers > 0:
                total_layers = len(self.backbone.encoder.layer)
                layers_to_unfreeze = self.backbone.encoder.layer[-unfreeze_last_n_layers:]
                for layer in layers_to_unfreeze:
                    for param in layer.parameters():
                        param.requires_grad = True
                print(f"Unfroze last {unfreeze_last_n_layers} transformer layer(s) out of {total_layers}")

        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size, 512),
            nn.LayerNorm(512), 
            nn.GELU(),
            nn.Dropout(p=dropout_rate),

            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(p=dropout_rate * 0.7),

            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(p=dropout_rate * 0.5),

            nn.Linear(128, num_classes),
        )

    def forward(self, x):
         # Resize to 224x224 if needed (Phikon requires this size)
        if x.shape[-1] != 224 or x.shape[-2] != 224:
            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)

        outputs = self.backbone(x)
        cls_token = outputs.last_hidden_state[:, 0] 
        return self.classifier(cls_token)

In [ ]:
class EfficientNetB0TransferLearning(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.3, freeze_backbone=True):
        super().__init__()
        self.backbone = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
        )

        if freeze_backbone:
            for param in self.backbone.features.parameters():
                param.requires_grad = False
            # Unfreeze GroupNorm/BatchNorm for domain adaptation
            for module in self.backbone.features.modules():
                if isinstance(module, (nn.BatchNorm2d, nn.GroupNorm)):
                    for param in module.parameters():
                        param.requires_grad = True

        # Replace classifier with more regularization
        in_features = self.backbone.classifier[-1].in_features
        # NOTE: MPS doesn't support inplace
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate, inplace=False),
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=False),
            nn.Dropout(p=dropout_rate * 0.5, inplace=False),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        return self.backbone(x)

### Load Data and Setup

In [ ]:
# Setup directories and paths
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import glob
from collections import defaultdict

base_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
current_dir = os.path.join(base_dir, "preprocessed")
dataset_dir = os.path.join(base_dir, "dataset")

print(f"Data directory: {current_dir}")
print(f"Dataset directory: {dataset_dir}")

In [ ]:
# Function to load patches from folder

def load_patches_from_folder(folder, max_workers=None):
    """Load patches grouped by image ID.
    
    For files named img_XXXX_patch_YY.png, groups patches by image ID (XXXX).
    Returns list of lists: [[patches for img 0], [patches for img 1], ...]
    """
    filenames = sorted(glob.glob(os.path.join(folder, 'img_*.png')), 
                       key=lambda x: (int(os.path.basename(x).split('_')[1]), 
                                      int(os.path.basename(x).split('_')[3].split('.')[0])))
    
    def load_image(file):
        img = cv2.imread(file)
        return img.astype(np.float32) / 255.0
    
    # Use ThreadPoolExecutor for parallel image loading
    if max_workers is None:
        max_workers = min(8, (os.cpu_count() or 4))
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        all_patches = list(executor.map(load_image, filenames))
    
    # Group patches by image ID
    image_patches = defaultdict(list)
    for filepath, patch in zip(filenames, all_patches):
        img_id = int(os.path.basename(filepath).split('_')[1])
        image_patches[img_id].append(patch)
    
    # Convert to sorted list of lists
    sorted_ids = sorted(image_patches.keys())
    patches = [image_patches[img_id] for img_id in sorted_ids]
    
    return patches, sorted_ids

# Load test data
test_patches, test_imageIndeces = load_patches_from_folder(current_dir + '/test_data')

print(f"Loaded {len(test_patches)} test images")

In [ ]:
# Define constants and label mapping
BATCH_SIZE = 32
num_classes = 4

label_mapping = {
    'Luminal A': 0,
    'Luminal B': 1,
    'HER2(+)': 2,
    'Triple negative': 3
}

print(f"Label mapping: {label_mapping}")
print(f"Number of classes: {num_classes}")
print(f"Batch size: {BATCH_SIZE}")

### Load Pre-trained Model

In [ ]:
# Create model instance and load pre-trained weights
b0_model = EfficientNetB0TransferLearning(
    num_classes=num_classes, 
    dropout_rate=0.65, 
    freeze_backbone=True,
).to(device)

# Load the pre-trained weights
model_path = 'ensemble_models/efficientnetb0_tl_frozen_model.pt'
b0_model.load_state_dict(torch.load(model_path, map_location=device))
b0_model.eval()  # Set to evaluation mode

print(f"Model loaded from: {model_path}")
print(f"Model set to evaluation mode")

# Count parameters
total_params = sum(p.numel() for p in b0_model.parameters())
trainable_params = sum(p.numel() for p in b0_model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
phikon_model = PhikonTransferLearning(
    num_classes=num_classes, 
    dropout_rate=0.5, 
    freeze_backbone=True,
    unfreeze_last_n_layers=0,
).to(device)

# Load the pre-trained weights
model_path = 'ensemble_models/phikon_tl_frozen_model.pt'
phikon_model.load_state_dict(torch.load(model_path, map_location=device))
phikon_model.eval()  # Set to evaluation mode

print(f"Model loaded from: {model_path}")
print(f"Model set to evaluation mode")

# Count parameters
total_params = sum(p.numel() for p in phikon_model.parameters())
trainable_params = sum(p.numel() for p in phikon_model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
#Predict function for ensemble models
def predict_ensemble(models, normalize, test_patches, batch_size=BATCH_SIZE):
    """Predict classes for test patches using ensemble of models.
    
    Args:
        models: list of trained PyTorch models
        normalize: array of normalization method for dataset
        test_patches: list of lists of patches for each test image
        batch_size: batch size for DataLoader
    Returns:
        predictions: numpy array of predicted class indices for each test image
    """

    all_model_preds = []

    for i, model in enumerate(models):
        model_preds = []
        for patches in test_patches:
            dataset = AugmentDataset(
                data=np.array(patches), 
                labels=np.zeros(len(patches)),  # Dummy labels
                augmentation=None,
                normalize=normalize[i]
            )
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

            patch_preds = []
            with torch.no_grad():
                for inputs, _ in dataloader:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    probs = F.softmax(outputs, dim=1)
                    patch_preds.append(probs.cpu().numpy())
            
            patch_preds = np.vstack(patch_preds)
            avg_patch_pred = np.mean(patch_preds, axis=0)
            model_preds.append(avg_patch_pred)
        
        all_model_preds.append(np.vstack(model_preds))
    
    # Average predictions from all models
    final_preds = np.mean(np.array(all_model_preds), axis=0)
    predicted_classes = np.argmax(final_preds, axis=1)

    return predicted_classes

In [ ]:
final_test_preds_tta = predict_ensemble([phikon_model, b0_model], normalize=[False, "imagenet"], test_patches=test_patches)

# Create inverse mapping from class index to label name
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

submission_df_tta = pd.DataFrame({
    'sample_index': [f'img_{test_imageIndeces[i]:04d}.png' for i in range(len(final_test_preds_tta))],
    'label': [inverse_label_mapping[pred] for pred in final_test_preds_tta]
})

submission_df_tta.to_csv('submission.csv', index=False)

print(f"\\nTTA submission file created: submission.csv")
print(f"Prediction distribution with TTA:")
print(submission_df_tta['label'].value_counts())

### Test time augmentation
Test time augmentation (TTA) is a technique used in machine learning, particularly in computer vision, to improve the performance of models during inference. The idea is to apply various transformations to the input data at test time, generate multiple predictions for each transformed input, and then aggregate these predictions to obtain a final result.

In [ ]:
def predict_with_tta(model, test_patches, num_tta=5, normalize=True):
    """
    Generate predictions using Test-Time Augmentation.
    
    Args:
        model: Trained model
        test_patches: List of patch arrays per image
        num_tta: Number of augmented predictions to average (default: 5)
        use_normalization: Whether to apply ImageNet normalization (default: True)
    
    Returns:
        predictions: Array of predicted class indices
    """
    # TTA augmentations (mild transforms that preserve label)
    if normalize:
        tta_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        ])
    else:
        tta_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
        ])
    
    # Prepare test patches
    X_test_list = []
    image_idx_test_list = []
    for i, patches in enumerate(test_patches):
        for patch in patches:
            X_test_list.append(patch)
            image_idx_test_list.append(i)
    
    X_test_final = np.array(X_test_list)
    image_idx_test_final = np.array(image_idx_test_list)
    
    all_probs = []
    
    model.eval()
    with torch.no_grad():
        for tta_iter in range(num_tta):
            # Create dataset with TTA transforms
            dummy_labels = np.zeros(len(X_test_final), dtype=np.int64)
            
            if tta_iter == 0:
                # First iteration: no augmentation (original)
                if normalize:
                    test_ds = AugmentDataset(X_test_final, dummy_labels, augmentation=None, normalize="imagenet")
                else:
                    test_ds = AugmentDataset(X_test_final, dummy_labels, augmentation=None, normalize=None)
            else:
                # Subsequent iterations: apply TTA
                test_ds = AugmentDataset(X_test_final, dummy_labels, augmentation=tta_transforms, normalize=None)
            
            test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
            
            # Collect predictions for this TTA iteration
            tta_probs = []
            for xb, _ in test_loader:
                xb = xb.to(device)
                logits = model(xb)
                probs = F.softmax(logits, dim=1).cpu().numpy()
                tta_probs.append(probs)
            
            all_probs.append(np.concatenate(tta_probs))
            print(f"TTA iteration {tta_iter + 1}/{num_tta} completed")
    
    # Average probabilities across all TTA iterations
    avg_tta_probs = np.mean(all_probs, axis=0)
    
    # Aggregate by image (average across patches)
    image_probs = defaultdict(list)
    for i, (probs, img_i) in enumerate(zip(avg_tta_probs, image_idx_test_final)):
        image_probs[img_i].append(probs)
    
    final_preds = []
    for img_i in sorted(image_probs.keys()):
        avg_probs = np.mean(image_probs[img_i], axis=0)
        final_preds.append(avg_probs.argmax())
    
    return np.array(final_preds)

In [ ]:
# Generate predictions with TTA (set num_tta=1 to disable TTA)
USE_TTA = True
NUM_TTA = 5 if USE_TTA else 1

print(f"Generating predictions with TTA (num_tta={NUM_TTA})...")
final_test_preds_tta = predict_with_tta(phikon_model, test_patches, num_tta=NUM_TTA, normalize=False)

# Create inverse mapping from class index to label name
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

submission_df_tta = pd.DataFrame({
    'sample_index': [f'img_{test_imageIndeces[i]:04d}.png' for i in range(len(final_test_preds_tta))],
    'label': [inverse_label_mapping[pred] for pred in final_test_preds_tta]
})

submission_df_tta.to_csv('submission_tta.csv', index=False)

print(f"\\nTTA submission file created: submission_tta.csv")
print(f"Prediction distribution with TTA:")
print(submission_df_tta['label'].value_counts())